<a href="https://colab.research.google.com/github/Kiron-Ang/DFHL/blob/main/cortisol_bdi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Print Python version and install/update/import libraries!
!python -V

!pip install -U scikit-learn > output.txt
import sklearn
print("scikit-learn", sklearn.__version__)

!pip install -U matplotlib > output.txt
import matplotlib
print("matplotlib", matplotlib.__version__)

!pip install -U seaborn > output.txt
import seaborn
print("seaborn", seaborn.__version__)

!pip install -U polars > output.txt
import polars
print("polars", polars.__version__)

# Read in two CSV files, one with cortisol data and one with BDI data
# The BDI data file is named "prediction" because the goal is to extrapolate
# and predict the salivary cortisol measurement of the people to draw correlations.
# This must be done because the cortisol is from 1989 to 1999,
# while the BDI scores are from July 2003.
cortisol = polars.read_csv("cortisol.csv")
print("cortisol", cortisol.shape)
prediction = polars.read_csv("prediction.csv")
print("prediction", prediction.shape)

Python 3.10.12
scikit-learn 1.5.2
matplotlib 3.7.1
seaborn 0.13.2
polars 1.8.2
cortisol (21367, 12)
prediction (403, 13)
